In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import os
import cv2
from PIL import Image
import pickle
import imutils

In [3]:
#Build training images
path = 'CharSplit/'
#Paste the images onto a constant background and blend the images    
def setImage(image):
    image = imutils.resize(image,height=20)
    gray_masked = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    (thresh, im_bw) = cv2.threshold(gray_masked, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #print(im_bw.shape)
    #Draw a white canvas
    white_img = np.zeros((40,40), np.uint8)
    white_img[:,:] = (255)
    #Paste the image onto the white canvas
    x_offset=y_offset=1
    white_img[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = im_bw
    return white_img

for filename in os.listdir(path+'0/'):
    org_img = cv2.imread(path+'0/'+filename)    
    #r = 15.0 / org_img.shape[1]
    #dim = (15, int(org_img.shape[0] * r))
    # perform the actual resizing of the image and show it
    #resized = cv2.resize(org_img, dim, interpolation = cv2.INTER_AREA) 
    cv2.imwrite('CharSplitTrain/0/'+filename,setImage(org_img))
    
    
print('Training images created for 0')

for filename in os.listdir(path+'1/'):
    org_img = cv2.imread(path+'1/'+filename)    
    #r = 15.0 / org_img.shape[1]
    #dim = (15, int(org_img.shape[0] * r))
    # perform the actual resizing of the image and show it
    #resized = cv2.resize(org_img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite('CharSplitTrain/1/'+filename,setImage(org_img))   
    
print('Training images created for 1')


Training images created for 0
Training images created for 1


In [90]:
noOfFiles0 = len([name for name in os.listdir('CharSplitTrain/0/')])
noOfFiles1 = len([name for name in os.listdir('CharSplitTrain/1/')])
noOfFiles = noOfFiles0 + noOfFiles1

print('Total number of files = '+str(noOfFiles))
reshaped = np.empty([noOfFiles,1600])
Y = np.empty([noOfFiles,1])


j=0
for filename in os.listdir('CharSplitTrain/0/'):    
    img = cv2.imread('CharSplitTrain/0/'+filename,0)
    reshaped[j]=img.reshape(1,1600)
    Y[j] = 0
    j=j+1
print('loaded from 0')
for filename in os.listdir('CharSplitTrain/1/'):
    img = cv2.imread('CharSplitTrain/1/'+filename,0)
    reshaped[j]=img.reshape(1,1600)
    Y[j] = 1
    j=j+1
print('loaded from 1')   


Total number of files = 366
loaded from 0
loaded from 1


In [91]:
Y = np.ravel(Y)
(X_Train,X_Test,Y_Train,Y_Test) = train_test_split(reshaped,Y)
model2 = LogisticRegression(C=1e5)
model2.fit(X_Train, Y_Train)
predicted = model2.predict(X_Test)
print(predicted)
#probs = model2.predict_proba(X_Test)
#print (probs)
print (metrics.classification_report(Y_Test, predicted))


[ 0.  1.  1.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.
  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.
  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.
  0.  0.]
             precision    recall  f1-score   support

        0.0       0.88      0.85      0.87        62
        1.0       0.72      0.77      0.74        30

avg / total       0.83      0.83      0.83        92



In [92]:
pickle.dump(model2,open('CharSplit.sav','wb'))